In [ ]:
from newsdataapi import NewsDataApiClient
import csv
import os

# Định nghĩa lớp NewsAPI

In [ ]:
class  NewsApiClient:
       def __init__(self):
           self.api = NewsDataApiClient(apikey='pub_19636e9e4488149d9bee3c8c740a8c89eeb4b')
       def extract_information_from_api(self , country, category):
            page=None
            self.results = []
            while True:
                response = self.api.news_api(country , category , page = page)
                self.results += response['results']
                page = response.get('nextPage',None)
                if not page:
                    break
            return self.results

In [ ]:
newsgetter = NewsApiClient()

# Cào dữ liệu từ API 

In [ ]:
result_sport  = newsgetter.extract_information_from_api('vi', 'sports') #thể thao

In [ ]:
result_technology = newsgetter.extract_information_from_api('vi', 'technology') #công nghệ

In [ ]:
result_bussiness = newsgetter.extract_information_from_api('vi' , 'business') #kinh doanh

In [ ]:
result_science = newsgetter.extract_information_from_api('vi' , 'science') #khoa học

In [ ]:
result_entertainment = newsgetter.extract_information_from_api('vi' , 'entertainment') #giải trí

In [ ]:
result_health = newsgetter.extract_information_from_api('vi', 'health') #sức khoẻ

In [ ]:
result_world = newsgetter.extract_information_from_api('vi', 'world') #thế giới

In [ ]:
result_environment = newsgetter.extract_information_from_api('vi' , 'environment') #môi trường

In [ ]:
result_top = newsgetter.extract_information_from_api('vi' , 'top') #Top bài báo

# Tinh chỉnh và lưu trữ dữ liệu

In [ ]:
#tổ chức cột dữ liệu: idx , title , description , link , source, category , country , language , publishdate
field_names= ['id' , 'title' , 'link' ,'description','pubDate',  'source_id', 'category' , 'country' , 'language']
list_keys_remove = ['creator','image_url', 'content', 'video_url', 'keywords']

In [ ]:
def process_1(news):
    for i in range(len(news)):
        for key in list_keys_remove:
            news[i].pop(key)
    return news
def process_2(news):    
    news_updated = []
    for i in range(len(news)):
        temp ={}
        for feature in field_names:
            if feature == 'id':
                 temp[feature] = i + 1
            elif feature == 'category' or feature =='country':
                 temp[feature] = news[i][feature][0]
            else:
                 temp[feature] = news[i][feature]
        news_updated.append(temp)
    return news_updated
def process_3(news , length = 0):
    news_updated = []
    for i in range(len(news)):
        temp ={}
        for feature in field_names:
            if feature == 'id':
                 temp[feature] = i + 1 + length
            elif feature == 'category' or feature =='country':
                 temp[feature] = news[i][feature][0]
            else:
                 temp[feature] = news[i][feature]
        news_updated.append(temp)
    return news_updated
    

In [ ]:
news = result_sport + result_bussiness + result_entertainment + result_environment + result_science + result_technology + result_top + result_world  + result_health

In [ ]:
import pandas as pd
if os.path.exists('NewsVN.csv'):
    newsVN_csv = pd.read_csv('NewsVN.csv')
    length = newsVN_csv.shape[0]
    news_updated = process_3(process_1(news) , length)
else:
    news_updated = process_2(process_1(news))

In [ ]:

if os.path.exists('NewsVN.csv'):
    new_path = open("NewsVN.csv", "a" , encoding = "UTF-8")
    z = csv.writer(new_path)
    for d in news_updated:
        z.writerow(d.values())
    new_path.close()
else:
    with open('NewsVN.csv', 'w', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile , field_names = field_names)
        writer.writeheader()
        writer.writerows(news_updated)

# Cập nhật dữ liệu

In [ ]:
import pandas as pd
raw_data = pd.read_csv('NewsVN.csv')
raw_data

In [ ]:
#xử lý đường dẫn giống nhau
unique_url_data = raw_data['link'].unique()
url_data = raw_data['link']
dict_id_url = {}
for url in unique_url_data:
    temp = []
    for i in range(len(url_data)):
        if url == url_data[i]:
            temp.append(i+1)
    dict_id_url[url] = temp
           

In [ ]:
list_unique_id = [sample[len(sample)-1] for sample in list(dict_id_url.values())]

In [ ]:
list_unique_id.sort(reverse=False)

In [ ]:
new_data = pd.DataFrame(data= list_unique_id, columns=['id'])

In [ ]:
processed_data = new_data.merge(raw_data)

In [ ]:
processed_data

In [ ]:
for i in range(0 , processed_data.shape[0]):
     processed_data.at[i, 'id'] = i+1

In [ ]:
processed_data

In [ ]:
processed_data.to_csv('NewsVN.csv' ,sep=',', columns=['id' , 'title','link', 'description','pubDate','source_id','category','country','language'] , index=None)